In [1]:
!pip -q install tensorflow-text

     |████████████████████████████████| 3.0MB 8.5MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive')
!nvidia-smi

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
Mon May 18 03:43:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======

In [0]:
!cp "/gdrive/My Drive/bili-filter/comments_clip.txt" "comments.txt"
!cp "/gdrive/My Drive/bili-filter/comments.txt" "comments_test.txt"
!cp "/gdrive/My Drive/bili-filter/target_clip.txt" "targets.txt"

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
import os
from collections import Counter
import time
import random

In [0]:
# #blackmagic 
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [6]:
with open("comments.txt","r",encoding="utf-8") as fin:
    comments = fin.read()
    comments = comments.split()

with open("targets.txt","r",encoding='utf-8') as fin:
    targets = fin.read()
    targets = [int(x) for x in targets.split()]

print("Total:",len(comments),"comments")

Total: 1326 comments


In [0]:
embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3", input_shape=[], dtype=tf.string, trainable=False)

def build_model(hidden_units=128, output_units=1):
    
    model = tf.keras.models.Sequential([
        #tf.keras.layers.Input(shape=(1,), dtype=tf.string),
        #tf.keras.layers.Lambda(tf.squeeze),
        embed,
        tf.keras.layers.Dense(hidden_units, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(output_units, activation=tf.keras.activations.sigmoid)
    ])
    return model


In [8]:
model = build_model(64,1)
model.load_weights("/gdrive/My Drive/bili-filter/saves/model_weight")


Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7ff5678cf5f8> and <tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8e48>).



Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7ff5678cf5f8> and <tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8e48>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8e48> and <tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8c50>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8e48> and <tensorflow.python.keras.layers.core.Dense object at 0x7ff5686c8c50>).


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 512)               68927232  
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 68,960,129
Trainable params: 32,897
Non-trainable params: 68,927,232
_________________________________________________________________


In [0]:
dataset = tf.data.Dataset.from_tensor_slices( (comments, targets) )
#char_dataset = tf.data.Dataset.from_generator(data_gen, tf.int32)

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', input_example)
  print ('Target data:', target_example)

Input data:  tf.Tensor(b'\xe7\xba\xa2\xe7\x8b\xbc\xe6\x88\x98\xe6\xad\x8c', shape=(), dtype=string)
Target data: tf.Tensor(1, shape=(), dtype=int32)


In [0]:
BATCH_SIZE = 32
BUFFER_SIZE = 5000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.prefetch(3)

In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'] )

checkpoint_dir = '/gdrive/My Drive/bili-filter/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
# # if using TPU
# with strategy.scope():
#     model = build_model(
#         vocab_size = vocab_size,
#         embedding_dim = embedding_dim,
#         hidden_dim = hidden_dim,
#         batch_size = BATCH_SIZE,
#         drop_rate = 0.1
#     )

#     model.compile(optimizer='adam', loss="sparse_categorical_crossentropy")

#     checkpoint_dir = '/gdrive/My Drive/bili-filter/training_checkpoints'
#     # Name of the checkpoint files
#     checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

#     checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_prefix,
#         save_weights_only=True
#     )

In [14]:
EPOCHS = 25
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
41/41 [==============================] - 4s 91ms/step - loss: 0.6452 - accuracy: 0.6486
Epoch 2/25
41/41 [==============================] - 2s 61ms/step - loss: 0.6081 - accuracy: 0.7287
Epoch 3/25
41/41 [==============================] - 2s 41ms/step - loss: 0.5915 - accuracy: 0.7729
Epoch 4/25
41/41 [==============================] - 2s 59ms/step - loss: 0.5837 - accuracy: 0.8018
Epoch 5/25
41/41 [==============================] - 2s 49ms/step - loss: 0.5775 - accuracy: 0.8102
Epoch 6/25
41/41 [==============================] - 2s 57ms/step - loss: 0.5738 - accuracy: 0.8209
Epoch 7/25
41/41 [==============================] - 10s 256ms/step - loss: 0.5699 - accuracy: 0.8316
Epoch 8/25
41/41 [==============================] - 2s 58ms/step - loss: 0.5665 - accuracy: 0.8361
Epoch 9/25
41/41 [==============================] - 12s 283ms/step - loss: 0.5644 - accuracy: 0.8430
Epoch 10/25
41/41 [==============================] - 2s 56ms/step - loss: 0.5622 - accuracy: 0.8453
Epoch

In [18]:
print(model(tf.constant(["演如果不是演的呢也许厂长就是不知道可以跳或没想到呢"])))

tf.Tensor([[0.833735]], shape=(1, 1), dtype=float32)


In [21]:
with open("comments_test.txt","r",encoding="utf-8") as fin:
    comments_test = fin.read()
    comments_test = comments_test.split()

random.shuffle(comments_test)

for index, comment in enumerate(comments_test):
    if index > 100:
        break
    
    output = model(tf.constant([comment]))
    print(comment, output.numpy())

老鹅我支持你 [[0.9924716]]
zww加油加油加油把未来变得更好 [[0.99984765]]
11111合影 [[0.00022219]]
牛逼 [[0.00014546]]
哥们 [[0.9208279]]
死神来收人了 [[0.9999931]]
蕉迟但到 [[0.26017505]]
3连 [[0.0002072]]
卡了 [[0.00850832]]
ohhhhhhhhhh [[4.158056e-07]]
冷酷无情花少北 [[0.99905294]]
敖厂长万能的哥们 [[0.9898022]]
为了部落 [[0.99291784]]
粪黄传奇粪黄 [[0.1621512]]
为什么拿原版舞蹈骗我 [[0.38566384]]
老叶来啦 [[0.8173161]]
1316万的硬币 [[0.00041338]]
醋帝 [[0.99999523]]
RIP [[0.97541124]]
毛67 [[2.3255716e-06]]
AI好玩 [[0.99982613]]
掉血 [[0.79179716]]
镇馆之宝啊 [[0.99990416]]
AWSL [[0.98939437]]
哥们 [[0.9208279]]
兄弟会表示无fa说 [[0.99998796]]
预定厂长 [[0.87973386]]
败者食尘 [[0.9999453]]
70分 [[5.217229e-06]]
我在bilibili我很骄傲 [[1.]]
害怕 [[0.00115703]]
NB啊 [[9.623816e-06]]
笑死 [[0.02977275]]
不如鬼畜 [[0.7663941]]
但是bgm是真的好听 [[0.9999962]]
考古的弹幕礼仪忘了 [[0.766504]]
牛逼牛逼牛逼牛逼牛逼 [[0.00091217]]
我老丈人带我玩的这个 [[0.9931611]]
WY特色 [[0.4182174]]
我粉了个什么玩意 [[0.01627373]]
哈哈哈哈哈哈哈 [[4.3317797e-07]]
66666666 [[1.7734068e-08]]
博爱好可爱啊 [[1.]]
现在不是2020吗 [[5.9667316e-05]]
前方高能预警 [[0.9766867]]
ohhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh